# Hashtag Counter
For each dataset, each dataset is stored in db-id, below are the related table

ret_available_db
ret_tweet

result stored in hashtag_counter with table structure of
id, int -> primary key
db_id, int -> foreign key to ret_available_db.db_id
tweet_id, int -> foreign to ret_tweet.id
hashtag_content, text -> the hashtag
hashtag_counter, int -> how many times hashtag is encountered
process_time, timestamp -> when the process is done


## First Step
query from database, that is not available in table hashtag_counter, take the last one first

In [6]:
import data_connector as dc
import re
import pandas as pd
import string
from collections import Counter

sql = """
select db_id from ret_available_db a  
where a.db_id not in (select db_id from hashtag_counter union select db_id from hashtag_processed) 
and count > 0
order by since desc 
limit 1
"""

df_db_id = dc.execute_query_psql(sql)
const_dbid = df_db_id.iloc[0]['db_id']

In [7]:
print(const_dbid)

87381


In [8]:
sql = """
select  *
from 	ret_tweet a 
where 	a.db_id = %s
"""
# sql = sql % "87546"
sql = sql % const_dbid
df_source = dc.execute_query_psql(sql)

#
# handling zero rows dataset
if len(df_source) == 0:
    sql = """
        INSERT INTO public.hashtag_processed
        (db_id, process_time)
        VALUES( %s, NOW());"""
    sql = sql % (const_dbid)
    print("recording db id %s as not existen" % const_dbid)
    dc.execute_query_psql(sql)
    quit()

In [9]:
import re

def expand_hashtag(text):
    hashtags = []
    # print(text)
    for match in re.findall(r'#(\w+)', text):
        # print(match)
        hashtags.append(match)
    return hashtags

# define output column
column_names = ['db_id','hashtag','tweet_id']

# Create a blank DataFrame with predefined columns
df_hashtag = pd.DataFrame(columns=column_names)

for index in range(len(df_source)):
    strTweetString = str(df_source['tweet'].iloc[index])
    # check if string is empty
    if strTweetString.strip() != "":
        list_res = expand_hashtag(strTweetString)
        # check if no hashtag found
        if len(list_res) > 0:
            # print(list_res)    
            for value in list_res:
                new_row = { 'db_id' : const_dbid,
                            'hashtag' : value,
                            'tweet_id' : df_source['id'].iloc[index] }
                # print(new_row)
                df_hashtag.loc[len(df_hashtag)+1] = new_row



## Record the result
into table hashtags_counter

In [ ]:
for i in range(len(df_hashtag)):
    hashtag = df_hashtag.iloc[i]['hashtag']
    # count = df_hashtag.iloc[i]['Count']
    tweet_id = df_hashtag.iloc[i]['tweet_id']

    dc.execute_query_psql(
        """
        INSERT INTO public.hashtag_counter (db_id, hashtag_content, tweet_id, process_time)
        VALUES (%s, '%s', %s, NOW())
        """ % 
        (const_dbid, hashtag, tweet_id)
    )

In [13]:
len(df_hashtag)

80034

In [5]:
# wait 10 seconds before finished
import time
time.sleep(10)